In [6]:
!pip install yfinance

In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from datetime import datetime

# Load Excel
df = pd.read_excel('/content/Daftar Saham  - 20250523.xlsx')

# Pastikan kolom 'Saham' numeric dan drop data invalid
df['Saham'] = pd.to_numeric(df['Saham'], errors='coerce')
df_shares = df[['Kode', 'Saham']].dropna(subset=['Saham'])

# Buat dict kode saham -> jumlah saham
market_caps_shares = dict(zip(df_shares['Kode'].astype(str), df_shares['Saham']))

# List ticker buat yfinance
tickers = df['Kode'].dropna().astype(str) + '.JK'
tickers = tickers.tolist()

def compute_stoch_rsi(close, rsi_length=14, stoch_length=14, k=3, d=3):
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=rsi_length).mean()
    avg_loss = loss.rolling(window=rsi_length).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    min_rsi = rsi.rolling(window=stoch_length).min()
    max_rsi = rsi.rolling(window=stoch_length).max()
    stoch_rsi = (rsi - min_rsi) / (max_rsi - min_rsi)
    k_line = stoch_rsi.rolling(window=k).mean() * 100
    d_line = k_line.rolling(window=d).mean()
    return k_line, d_line

def screen_stock(ticker):
    try:
        score = 0
        ticker_code = ticker.replace('.JK', '')

        shares_outstanding = market_caps_shares.get(ticker_code, None)
        if shares_outstanding is None:
            return None

        ticker_obj = yf.Ticker(ticker)
        hist = ticker_obj.history(period="1d")
        if hist.empty:
            return None
        last_price = hist['Close'].iloc[-1]

        market_cap = shares_outstanding * last_price

        # Filter market cap minimal
        if market_cap < 5_000_000_000_000:
            return None

        df_hist = ticker_obj.history(period="6mo", interval="1d")
        if df_hist.empty or len(df_hist) < 50:
            return None

        df_hist['SMA20'] = df_hist['Close'].rolling(window=20).mean()
        df_hist['K'], df_hist['D'] = compute_stoch_rsi(df_hist['Close'])
        ema_12 = df_hist['Close'].ewm(span=12, adjust=False).mean()
        ema_26 = df_hist['Close'].ewm(span=26, adjust=False).mean()
        df_hist['MACD'] = ema_12 - ema_26
        df_hist['MACD_signal'] = df_hist['MACD'].ewm(span=9, adjust=False).mean()

        latest = df_hist.iloc[-1]
        prev = df_hist.iloc[-2]

        if latest['Close'].item() <= latest['SMA20'].item():
            return None

        conditions_met = []

        if (prev['K'].item() < prev['D'].item()) and (latest['K'].item() >= latest['D'].item()) and (latest['K'].item() < 50):
            score += 4
            conditions_met.append("StochRSI K cross D & K<50")

        if (prev['K'].item() < 0) and (latest['K'].item() >= 0):
            score += 3
            conditions_met.append("StochRSI K cross up 0")

        if (prev['MACD'].item() < 0) and (latest['MACD'].item() >= 0):
            score += 4
            conditions_met.append("MACD level cross up 0")

        if (prev['MACD'].item() < prev['MACD_signal'].item()) and (latest['MACD'].item() >= latest['MACD_signal'].item()):
            score += 5
            conditions_met.append("MACD cross signal")

        if score > 0:
            return {
                "Kode": ticker_code,
                "Last Price": last_price,
                "MarketCap": market_cap,
                "Score": score,
                "Keterangan": ", ".join(conditions_met)
            }

    except Exception as e:
        print(f"❌ Error di {ticker}: {e}")
        return None

    return None

def send_telegram_message(token, chat_id, message):
    url = f"https://api.telegram.org/bot7487407302:AAGe6I8fLFGt19BfU7AFh31YMHaFVHUmu7U/sendMessage"
    data = {"chat_id": 652946372, "text": message}
    response = requests.post(url, data=data)
    if response.status_code == 200:
        print("✅ Notifikasi Telegram terkirim!")
    else:
        print(f"❌ Gagal kirim notifikasi: {response.text}")

def format_telegram_message(df_result, max_items=25):
    today = datetime.now().strftime("%d %B %Y")
    message = f"Ini rekomendasi Saham Buat kamu Hari ini {today}:\n"

    for i, row in df_result.head(max_items).iterrows():
        kode = row['Kode']
        score = row['Score']
        keterangan = row['Keterangan']
        message += f"{i+1}. {kode}, {score}. {keterangan}\n"
    return message

results = []
for t in tickers:
    res = screen_stock(t)
    if res is not None:
        results.append(res)

if len(results) == 0:
    print("❗ Tidak ada saham yang memenuhi kriteria screening")
else:
    df_result = pd.DataFrame(results)
    df_result = df_result.sort_values(by=["Score", "MarketCap"], ascending=[False, False])
    df_result.reset_index(drop=True, inplace=True)
    display(df_result)

    # Kirim notif Telegram
    BOT_TOKEN = "bot7487407302:AAGe6I8fLFGt19BfU7AFh31YMHaFVHUmu7U"
    CHAT_ID = 652946372  # ganti dengan chat_id mu
    pesan = format_telegram_message(df_result)
    send_telegram_message(BOT_TOKEN, CHAT_ID, pesan)


ERROR:yfinance:$ABDA.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$ALMI.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$ARTI.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$BCIC.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$BIKA.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$BLTZ.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$BTEL.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$CNTX.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$COWL.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$DEFI.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$ETWA.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$FASW.JK: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$FISH.JK: possibly delisted; no price data found  (period=1d)

,Kode,Last Price,MarketCap,Score,Keterangan
0,MAPI,1345.0,2.232700e+13,5,MACD cross signal
1,PSAB,310.0,8.202600e+12,5,MACD cross signal
2,SHIP,2080.0,5.657163e+12,5,MACD cross signal
3,BREN,6575.0,8.796444e+14,4,StochRSI K cross D & K<50
4,UNTR,22025.0,8.215623e+13,4,StochRSI K cross D & K<50
5,ISAT,2100.0,6.772670e+13,4,StochRSI K cross D & K<50
6,BRMS,384.0,5.444507e+13,4,StochRSI K cross D & K<50
7,NCKL,730.0,4.606198e+13,4,StochRSI K cross D & K<50
8,BNGA,1805.0,4.492786e+13,4,StochRSI K cross D & K<50
9,MBMA,362.0,3.909434e+13,4,StochRSI K cross D & K<50


✅ Notifikasi Telegram terkirim!
